In [30]:
from pyspark import SparkContext
from pyspark.sql.types import StructField, StructType, IntegerType, StringType, FloatType
from pyspark.sql import SQLContext
from pyspark.sql.functions import udf

In [9]:
spark = SparkContext(master='local', appName='Dataframes')


In [27]:
sqlContext = SQLContext(spark)

In [8]:
#spark.stop()

In [5]:
path = "files/"

In [4]:
!head -n 5 files/deportistaError.csv

deportista_id,nombre,genero,edad,altura,peso,equipo_id
1,A Dijiang,1,24,180,80,199
2,A Lamusi,1,23,170,60,199
3,Gunnar Nielsen Aaby,1,24,,,273
4,Edgar Lindenau Aabye,1,34,,,278


In [11]:
deportistaError = spark.textFile(path+'deportistaError.csv') \
    .map(lambda l : l.split(','))

In [12]:
def eliminaEncabezado(indice, iterador):
    return iter(list(iterador)[1:])

In [13]:
deportistaError =  deportistaError.mapPartitionsWithIndex(eliminaEncabezado)

In [15]:
deportistaError.take(3)

[['1', 'A Dijiang', '1', '24', '180', '80', '199'],
 ['2', 'A Lamusi', '1', '23', '170', '60', '199'],
 ['3', 'Gunnar Nielsen Aaby', '1', '24', '', '', '273']]

In [19]:
deportistaError = deportistaError.map(lambda l:(
l[0],
l[1],
l[2],
l[3],
l[4],
l[5],
l[6]
)
)

In [25]:
schema = StructType(
    [
        StructField('deportista_id', StringType(), False),
        StructField('nombre', StringType(), False),
        StructField('genero', StringType(), False),
        StructField('edad', StringType(), False),
        StructField('altura', StringType(), False),
        StructField('peso', StringType() , False),
        StructField('equipo_id', StringType(),False)
    ]
)

In [28]:
deportistaErrorDF = sqlContext.createDataFrame(deportistaError, schema)

In [29]:
deportistaErrorDF.show()

+-------------+--------------------+------+----+------+----+---------+
|deportista_id|              nombre|genero|edad|altura|peso|equipo_id|
+-------------+--------------------+------+----+------+----+---------+
|            1|           A Dijiang|     1|  24|   180|  80|      199|
|            2|            A Lamusi|     1|  23|   170|  60|      199|
|            3| Gunnar Nielsen Aaby|     1|  24|      |    |      273|
|            4|Edgar Lindenau Aabye|     1|  34|      |    |      278|
|            5|Christine Jacoba ...|     2|  21|   185|  82|      705|
|            6|     Per Knut Aaland|     1|  31|   188|  75|     1096|
|            7|        John Aalberg|     1|  31|   183|  72|     1096|
|            8|"Cornelia ""Cor""...|     2|  18|   168|    |      705|
|            9|    Antti Sami Aalto|     1|  26|   186|  96|      350|
|           10|"Einar Ferdinand ...|     1|  26|      |    |      350|
|           11|  Jorma Ilmari Aalto|     1|  22|   182|76.5|      350|
|     

In [31]:
def conversionEnteros(valor):
    return int(valor) if len(valor)>0 else None

In [32]:
conversionEnteros_udf = udf(lambda z: conversionEnteros(z), IntegerType())

In [33]:
sqlContext.udf.register('conversionEnteros_udf', conversionEnteros_udf)

<function __main__.<lambda>(z)>

In [35]:
deportistaErrorDF.select(conversionEnteros_udf('altura').alias('alturaUDF')).show()

+---------+
|alturaUDF|
+---------+
|      180|
|      170|
|     null|
|     null|
|      185|
|      188|
|      183|
|      168|
|      186|
|     null|
|      182|
|      172|
|      159|
|      171|
|     null|
|      184|
|      175|
|      189|
|     null|
|      176|
+---------+
only showing top 20 rows

